## Weather Cleaning Notebook ##


Schoeggl_Vorauer 2020

cleaning weather data

datasetlink: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

In [2]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter 
import swifter # swifter is for multiprocessing pandas apply functions
from Function_Definitions import *

In [74]:
# import weatherdataset as csv
weather = pd.read_csv(r'..\Data\Raw\weatherAUS.csv')

In [62]:
# keyelements(columnnames output)
weather.keys()

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [55]:
# checking which columns have missing values


Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
dtype: int64

In [53]:
for col in weather.columns:
    print(col + ' has ' + str(round((weather[col].isnull().sum() / weather.shape[0]) * 100, 2)) + '% missing values')

Date has 0.0% missing values
Location has 0.0% missing values
MinTemp has 0.45% missing values
MaxTemp has 0.23% missing values
Rainfall has 0.99% missing values
Evaporation has 42.79% missing values
Sunshine has 47.69% missing values
WindGustDir has 6.56% missing values
WindGustSpeed has 6.52% missing values
WindDir9am has 7.04% missing values
WindDir3pm has 2.66% missing values
WindSpeed9am has 0.95% missing values
WindSpeed3pm has 1.85% missing values
Humidity9am has 1.25% missing values
Humidity3pm has 2.54% missing values
Pressure9am has 9.86% missing values
Pressure3pm has 9.83% missing values
Cloud9am has 37.74% missing values
Cloud3pm has 40.15% missing values
Temp9am has 0.64% missing values
Temp3pm has 1.92% missing values
RainToday has 0.99% missing values
RISK_MM has 0.0% missing values
RainTomorrow has 0.0% missing values


In [54]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [75]:
# dropping columns because they have many missing values
weather.drop(['Sunshine', 'Evaporation', 'Pressure3pm', 'Pressure9am', 'WindDir9am', 'WindDir3pm'], 1, inplace=True)

In [76]:
# as a large portion of australia is desert, we assume the missing values in Cloud mean no clouds
weather['Cloud9am'].fillna(0, inplace=True)
weather['Cloud3pm'].fillna(0, inplace=True)

In [77]:
# filling the missing values in WindGustDir assuming that the wind most often comes from the same direction
top = weather['WindGustDir'].describe().top
weather['WindGustDir'].fillna(top, inplace=True)

In [78]:
# assuming missing values in Rainfall as no rain
weather['Rainfall'].fillna(0, inplace=True)

In [79]:
weather.dropna(subset=['Humidity3pm', 'Temp3pm', 'Temp9am', 'MinTemp', 'MaxTemp'], inplace=True)

In [80]:
weather.isna().sum()

Date                0
Location            0
MinTemp             0
MaxTemp             0
Rainfall            0
WindGustDir         0
WindGustSpeed    7217
WindSpeed9am      800
WindSpeed3pm      734
Humidity9am       129
Humidity3pm         0
Cloud9am            0
Cloud3pm            0
Temp9am             0
Temp3pm             0
RainToday        1095
RISK_MM             0
RainTomorrow        0
dtype: int64

**split appended names** **change seperator and join saperated names**

In [81]:
# call stringchange function from Function_Definitions file
weather = stringchange(weather)
weather = weather.replace({'Pearce R A A F': 'Pearce'})

In [82]:
# weather column assign Locations to new variable 
weatherloc = weather.Location.unique()
weatherloc = pd.DataFrame(weatherloc, columns=['Location']) 

In [83]:
# checking if the unique locations in weather match weatherlocs
np.all(weather.Location.unique() == weatherloc.Location.unique())

True

**georeferencing locations**

In [84]:
# applying geocoordinates to locations
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
gc = Nominatim(user_agent="fintu-blog-geocoding-python")
#short test
bla1 , bla2 = gc.geocode("Pears, Australia")[1]

In [85]:
# applying geocoordinates to locations / define ratelimiter
def rate_limited_geocode(query):
    geocode = RateLimiter(gc.geocode, min_delay_seconds=1)
    return geocode(query)
# geocode here is another variable than above
def geocodeb(row):
    lookup_query = row["Location"] + ", " + "Australia"
    lookup_result = rate_limited_geocode(lookup_query)
    return lookup_result
weatherloc["loctemp"] = weatherloc.apply(geocodeb, axis=1)

In [86]:
# making sure that it finds values to all locations
weatherloc.isna().sum()

Location    0
loctemp     0
dtype: int64

In [87]:
# loctemp contains all geo-informations given above. here we extract longitude and latitude information.
tempbla = [i[1] for i in weatherloc.loctemp]
weatherloc['latitude'] , weatherloc['longitude'] = np.array(tempbla).T
weatherloc = weatherloc.drop(['loctemp'],1)

**assign the coordiantes given from above function to locations in weather**

In [88]:
# filling function for longitude, latitude into correct rows
def reverse_filling(df1, df2):
    templatitude = np.zeros(df1.shape[0])
    templongitude = np.zeros(df1.shape[0])
    for i in df1.Location.unique():
        templatitude[df1.Location == i] = df2[df2.Location==i].latitude
        templongitude[df1.Location == i] = df2[df2.Location==i].longitude
    df1['latitude'] = templatitude
    df1['longitude'] = templongitude
    return df1

In [89]:
# mapping of latitude and longitude values in weatherloc to the corresponding locations in weather
weather = reverse_filling(weather, weatherloc)

In [19]:
# short test
weather.latitude

0        -36.080477
1        -36.080477
2        -36.080477
3        -36.080477
4        -36.080477
            ...    
142188   -25.345554
142189   -25.345554
142190   -25.345554
142191   -25.345554
142192   -25.345554
Name: latitude, Length: 142193, dtype: float64

In [90]:
weather.shape

(137710, 20)

In [91]:
# renaming the Date column to match with the name from the fire data 'acq_date'
weather = weather.rename(columns={'Date': 'acq_date'})

In [92]:
#set aqc_date to datetime
weather['acq_date'] = pd.to_datetime(weather['acq_date'])
weather.head()

,acq_date,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,latitude,longitude
0,2008-12-01,Albury,13.4,22.9,0.6,W,44.0,20.0,24.0,71.0,22.0,8.0,0.0,16.9,21.8,No,0.0,No,-36.080477,146.91628
1,2008-12-02,Albury,7.4,25.1,0.0,WNW,44.0,4.0,22.0,44.0,25.0,0.0,0.0,17.2,24.3,No,0.0,No,-36.080477,146.91628
2,2008-12-03,Albury,12.9,25.7,0.0,WSW,46.0,19.0,26.0,38.0,30.0,0.0,2.0,21.0,23.2,No,0.0,No,-36.080477,146.91628
3,2008-12-04,Albury,9.2,28.0,0.0,NE,24.0,11.0,9.0,45.0,16.0,0.0,0.0,18.1,26.5,No,1.0,No,-36.080477,146.91628
4,2008-12-05,Albury,17.5,32.3,1.0,W,41.0,7.0,20.0,82.0,33.0,7.0,8.0,17.8,29.7,No,0.2,No,-36.080477,146.91628


In [93]:
# adding the Year and Month as separate columns
weather = year_month(weather)

In [35]:
weather.isnull().sum()

acq_date             0
Location             0
MinTemp            637
MaxTemp            322
Rainfall             0
Evaporation      60843
WindGustDir          0
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
latitude             0
longitude            0
Year                 0
Month                0
dtype: int64

In [94]:
# adding a State column calculated from the longitude and latitude values
weather= get_state(weather)

In [95]:
# changing the WindGustDir column into angles
weather.WindGustDir.unique()

array(['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', 'ENE', 'SSE',
       'S', 'NW', 'SE', 'ESE', 'E', 'SSW'], dtype=object)

In [96]:
weather = weather.replace(to_replace = weather.WindGustDir.unique()[[12,13,10,2,15,0,1,11,3,7,8,14,6,5,4,9]], value =[360/16*i for i in range(16)])
weather.WindGustDir.unique()[[12,13,10,2,15,0,1,11,3,7,8,14,6,5,4,9]]

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5])

In [97]:
# changing Yes/No into True/False
weather = weather.replace(to_replace = ['No', 'Yes'], value =[False , True])

In [98]:
weather.RainTomorrow.unique()

array([False,  True])

In [100]:
# adding season column 
weather['season'] = ((weather.Month+3)%12 + 3)//3

In [101]:
weather.to_csv(r'..\Data\Clean\cweather.csv', index=False)